[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mitiau/DNABERT-Z/blob/main/ZDNA-prediction.ipynb)

# Install dependencies and define helper functions

In [9]:
!pip3 install transformers
!pip3 install biopython
!pip3 install torch
!pip3 install numpy
!pip3 install scipy
!pip3 install tqdm
!pip3 install io

/nix/store/kxkdrxvc3da2dpsgikn8s2ml97h88m46-bash-interactive-5.2-p15/bin/bash: line 1: pip3: command not found
/nix/store/kxkdrxvc3da2dpsgikn8s2ml97h88m46-bash-interactive-5.2-p15/bin/bash: line 1: pip3: command not found
/nix/store/kxkdrxvc3da2dpsgikn8s2ml97h88m46-bash-interactive-5.2-p15/bin/bash: line 1: pip3: command not found
/nix/store/kxkdrxvc3da2dpsgikn8s2ml97h88m46-bash-interactive-5.2-p15/bin/bash: line 1: pip3: command not found
/nix/store/kxkdrxvc3da2dpsgikn8s2ml97h88m46-bash-interactive-5.2-p15/bin/bash: line 1: pip3: command not found
/nix/store/kxkdrxvc3da2dpsgikn8s2ml97h88m46-bash-interactive-5.2-p15/bin/bash: line 1: pip3: command not found
/nix/store/kxkdrxvc3da2dpsgikn8s2ml97h88m46-bash-interactive-5.2-p15/bin/bash: line 1: pip3: command not found


In [10]:
import torch
from torch import nn
import transformers
from transformers import BertTokenizer, BertForTokenClassification
import numpy as np
from Bio import SeqIO
from io import StringIO, BytesIO
from tqdm import tqdm
import pickle
import scipy
from scipy import ndimage
import os

In [11]:
def seq2kmer(seq, k):
    kmer = [seq[x:x+k] for x in range(len(seq)+1-k)]
    return kmer

def split_seq(seq, length = 512, pad = 16):
    res = []
    for st in range(0, len(seq), length - pad):
        end = min(st+512, len(seq))
        res.append(seq[st:end])
    return res

def stitch_np_seq(np_seqs, pad = 16):
    res = np.array([])
    for seq in np_seqs:
        res = res[:-pad]
        res = np.concatenate([res,seq])
    return res

# Select model and parameters

In [12]:
model_name = 'HG kouzine' #@param ["HG chipseq", "HG kouzine", "MM chipseq", "MM kouzine"]
model_confidence_threshold = 0.5 #@param {type:"number"}
minimum_sequence_length = 10 #@param {type:"integer"}

In [13]:
if model_name == 'HG chipseq':
    model_id = '1VAsp8I904y_J0PUhAQqpSlCn1IqfG0FB'
    model_file_name = 'hg_chipseq.pytorch_model.bin'
elif model_name == 'HG kouzine':
    model_id = '1dAeAt5Gu2cadwDhbc7OnenUgDLHlUvkx'
    model_file_name = 'hg_kouzine.pytorch_model.bin'
elif model_name == 'MM curax':
    model_id = '1W6GEgHNoitlB-xXJbLJ_jDW4BF35W1Sd'
    model_file_name = 'mm_curax.pytorch_model.bin'
elif model_name == 'MM kouzine':
    model_id = '1dXpQFmheClKXIEoqcZ7kgCwx6hzVCv3H'
    model_file_name = 'mm_kouzine.pytorch_model.bin'

In [23]:
def gdown_wrapper(gdrive_id, file_path):
    if os.path.exists(file_path):
        print(file_path, gdrive_id, 'already exists')
        return
    
    # curl "https://drive.google.com/uc?id=${id}&export=download&confirm=ABCD" --verbose -L -o 
    gdrive_url = 'https://drive.google.com/uc?id={id}&export=download&confirm=ABCD'.format(id=gdrive_id)
    
    print(gdrive_url, file_path)
    
    !curl -L --progress-bar -o "{file_path}" "{gdrive_url}"

In [18]:
data_path = './6-new-12w-0'
model_data_path = './pytorch_models'
model_file_path = os.path.join(model_data_path, model_file_name)

In [19]:
meta_files = [
    ('10sF8Ywktd96HqAL0CwvlZZUUGj05CGk5', os.path.join(data_path, 'config.json')),
    ('16bT7HDv71aRwyh3gBUbKwign1mtyLD2d', os.path.join(data_path, 'special_tokens_map.json')),
    ('1EE9goZ2JRSD8UTx501q71lGCk-CK3kqG', os.path.join(data_path, 'tokenizer_config.json')),
    ('1gZZdtAoDnDiLQqjQfGyuwt268Pe5sXW0', os.path.join(data_path, 'vocab.txt')),
]

In [25]:
!mkdir $data_path
!mkdir $model_data_path

gdown_wrapper(model_id, model_file_path)

for meta_file in meta_files:
    gdown_wrapper(meta_file[0], meta_file[1])

mkdir: cannot create directory ‘./6-new-12w-0’: File exists
mkdir: cannot create directory ‘./pytorch_models’: File exists
./pytorch_models/hg_kouzine.pytorch_model.bin 1dAeAt5Gu2cadwDhbc7OnenUgDLHlUvkx already exists
./6-new-12w-0/config.json 10sF8Ywktd96HqAL0CwvlZZUUGj05CGk5 already exists
./6-new-12w-0/special_tokens_map.json 16bT7HDv71aRwyh3gBUbKwign1mtyLD2d already exists
./6-new-12w-0/tokenizer_config.json 1EE9goZ2JRSD8UTx501q71lGCk-CK3kqG already exists
./6-new-12w-0/vocab.txt 1gZZdtAoDnDiLQqjQfGyuwt268Pe5sXW0 already exists


In [16]:
!cp {model_file_path} {data_path}/pytorch_model.bin

In [17]:
tokenizer = BertTokenizer.from_pretrained(data_path)
model = BertForTokenClassification.from_pretrained(data_path)
is_cuda_available = torch.cuda.is_available()
print('cuda is ', '' if is_cuda_available else 'not ', 'available')
if is_cuda_available:
    model.cuda()
else:
    model.cpu()

cuda is  not  available


# Upload fasta files for prediction

In [2]:
from ipywidgets import widgets

class Files:
    def upload(_):
        uploader = widgets.FileUpload(
            multiple=True
        )
        display(uploader)
        
        return uploader

files = Files()

multiple files may be selected:

In [3]:
uploader = files.upload()

FileUpload(value=(), description='Upload', multiple=True)

In [8]:
"""
# for older ipywidgets versions
uploaded = {k: v['content'] for k, v in uploader.value.items()}
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
"""

uploaded = {v['name']: v['content'] for v in uploader.value}
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

User uploaded file "Tshz2.fa" with length 1378 bytes
User uploaded file "input_example.fasta" with length 2204 bytes
User uploaded file "Human Tshz2,promoter,exon 1.fa" with length 2130 bytes
User uploaded file "example_toy.fasta" with length 2629 bytes


# Predict and save raw outputs

In [144]:
output_path = 'output'

In [145]:
!mkdir "{output_path}"

In [148]:
out = []
for key in uploaded.keys():
    print(key)
    out.append(key)
    result_dict = {}
    for seq_record in SeqIO.parse(StringIO(BytesIO(uploaded[key]).read().decode('UTF-8')), 'fasta'):
        kmer_seq = seq2kmer(str(seq_record.seq).upper(), 6)
        seq_pieces = split_seq(kmer_seq)
        print(seq_record.name)
        out.append(seq_record.name)
        with torch.no_grad():
            preds = []
            for seq_piece in tqdm(seq_pieces):
                input_ids = torch.LongTensor(tokenizer.encode(' '.join(seq_piece), add_special_tokens=False))
                input_ids_unsqueezed = None
                if is_cuda_available:
                    input_ids_unsqueezed = input_ids.cuda().unsqueeze(0)
                else:
                    input_ids_unsqueezed = input_ids.cpu().unsqueeze(0)
                outputs = torch.softmax(model(input_ids_unsqueezed)[-1],axis = -1)[0,:,1]
                preds.append(outputs.cpu().numpy())
        result_dict[seq_record.name] = stitch_np_seq(preds)



        labeled, max_label = scipy.ndimage.label(result_dict[seq_record.name]>model_confidence_threshold)
        print('  start     end')
        out.append('  start     end')
        for label in range(1, max_label+1):
            candidate = np.where(labeled == label)[0]
            candidate_length = candidate.shape[0]
            if candidate_length>minimum_sequence_length:
                print('{:8}'.format(candidate[0]), '{:8}'.format(candidate[-1]))
                out.append('{:8}'.format(candidate[0]) + '{:8}'.format(candidate[-1]))

    with open(os.path.join(output_path, key + '.preds.pkl'),"wb") as fh:
      pickle.dump(result_dict, fh)
    print()

with open(os.path.join(output_path, 'text_predictions.txt'),"w") as fh:
    for item in out:
        fh.write("%s\n" % item)


Tshz2.fa
Human


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  3.68it/s]


  start     end

Human Tshz2,promoter,exon 1.fa
Human


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  4.04it/s]

  start     end
    1134     1186
    1196     1211
    1218     1249
    1265     1281
    1576     1612

